## 1. mongoDB 에 크롤릴 데이터 저장하기
* Database, collection 생성
    * DB : movie_db,   col : actor_col
* data/actors.json file을 읽어서 처리하기

In [2]:
# warning 감추기
import warnings
warnings.filterwarnings(action='ignore')

In [8]:
import pymongo
import json

# 1. connection 생성
conn = pymongo.MongoClient(host='localhost', port=27017)
print(conn)

# 2. database 생성
movie = conn.movie_db
print(movie)
print(movie.name)

# 3. collection 생성
actors = movie.actor_col
print(actors)
print(actors.name)

# 4. json file open
with open('data/actors.json','r',encoding='utf8') as file:
    json_data = json.load(file)
#     actors.insert_many(json_data) # 최초 한번만 해주기
    
# document count
print(actors.estimated_document_count())

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movie_db')
movie_db
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movie_db'), 'actor_col')
actor_col
1903


In [10]:
for doc in actors.find()[1900:]:
    print(doc)

{'_id': ObjectId('5f28b4b7a1b05d5764d6da30'), '배우이름': '박근수', '흥행지수': 0, '출연작': ['오늘도 평화로운'], '순위': 1903, '직업': '배우', '성별': '남'}
{'_id': ObjectId('5f28b4b7a1b05d5764d6da31'), '배우이름': '김승우', '흥행지수': 0, '출연작': ['개같은 날의 오후'], '순위': 1904, '직업': '배우', '생년월일': '1969-02-24', '성별': '남', '홈페이지': 'http://www.milkyboy.com/', '신장/체중': '180cm, 70kg', '학교': '수원대경영대학원 스포츠경영학 석사', '취미': '야구, 골프', '특기': '골프, 구기종목', '소속사': '웰메이드스타엠'}
{'_id': ObjectId('5f28b4b7a1b05d5764d6da32'), '배우이름': '윤지원', '흥행지수': 0, '출연작': ['걷기왕'], '순위': 1905, '직업': '배우', '생년월일': '1994-10-07', '성별': '여', '홈페이지': 'http://instagram.com/yuldion'}


### 1) sort

In [98]:
docs = actors.find({}).sort('배우이름')
docs = actors.find({}).sort('배우이름', pymongo.DESCENDING) # 내림차순
docs = actors.find({}).sort('배우이름', 1) # 오름차순
docs = actors.find({}).sort([('배우이름', 1),('순위',-1)])
for idx, actor in enumerate(docs[:10],1):
    print(idx, actor['배우이름'], actor['순위'])

1 RM 187
2 갈소원 200
3 감소현 901
4 감우성 1680
5 감우성 1673
6 감우성 1665
7 강경준 987
8 강계식 1181
9 강계열 1211
10 강구한 650


### 2. exists
* 필드 값이 존재하는 경우를 검색한다.


In [20]:
# '특기' 필드가 존재하는 경우와 존재하지 않는 경우 count
print(actors.find({'특기':{"$exists": True}}).count())
print(actors.find({'특기':{"$exists": False}}).count())

180
1723


In [99]:
# 생년월일이 있는 배우의 이름과 생년월일을 검색
doc = actors.find({},{'배우이름':1, '생년월일':1, '_id':0}) # 생년월일이 없는 필드는 조회못함
doc = actors.find({'생년월일':{'$exists': True}},{'배우이름':1, '생년월일':1, '_id':0})

for act in doc[:10]:
    print(act['배우이름'], act['생년월일'])

이병헌 1970-07-12
하정우 1978-03-11
마동석 1971-03-01
전혜진 1976-08-10
이성민 1968-10-15
김아중 1982-10-16
박정민 1987-02-25
강기영 1983-10-14
조진웅 1976-03-03
조정석 1980-12-26


In [34]:
# '다른이름' 이 '조원준'인 배우위 이름과 순위를 출력
doc = actors.find({'다른 이름': '조원준'},{'배우이름':1, '순위':1, '_id':0})
for i in doc:
    print(i['배우이름'], i['순위'])

조진웅 9


### 3) 필드의 범위로 검색
* $gt, $gte
* $lt, $lte

In [46]:
# 흥행지수 10000 <= 흥행지수 <= 30000
filter = {'흥행지수':{'$gte':10000, '$lte':30000}}
field = {'배우이름':1, '흥행지수':1,'출연작':1, '_id':0}
doc = actors.find(filter,field)
for i in doc:
    print(i)

{'배우이름': '마동석', '흥행지수': 19019, '출연작': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '전혜진', '흥행지수': 17976, '출연작': ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사']}
{'배우이름': '이성민', '흥행지수': 15213, '출연작': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '김아중', '흥행지수': 15208, '출연작': ['나쁜 녀석들: 더 무비', '더 킹']}
{'배우이름': '박정민', '흥행지수': 14534, '출연작': ['시동', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '동주']}
{'배우이름': '강기영', '흥행지수': 13974, '출연작': ['엑시트', '가장 보통의 연애', '퍼즐']}
{'배우이름': '조진웅', '흥행지수': 13472, '출연작': ['광대들: 풍문조작단', '퍼펙트맨', '블랙머니', '장수상회', '사라진 시간', '명량']}
{'배우이름': '조정석', '흥행지수': 13192, '출연작': ['엑시트', '형', '시간이탈자', '특종: 량첸살인기', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '배수지', '흥행지수': 12646, '출연작': ['백두산', '건축학개론']}
{'배우이름': '유아인', '흥행지수': 12422, '출연작': ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말']}
{'배우이름': '임윤아', '흥행지수': 12343, '출연작': ['엑시트']}
{'배우이름': '고두심', '흥행지수': 11496, '출연작': ['엑시트']}
{'배우이름': '장기용', '흥행지수': 11406, '출연작': ['나쁜 녀석들

## 4) or, nor(not or)

In [49]:
# 흥행지수 >= 15000 이거나 랭킹 <= 20인 배우 이름과 출연영화 리스트 출력
filter = {'$or':[{'흥행지수':{'$gte':15000}},{'랭킹':{'$lte':20}}]}
field = {'배우이름':1, '출연작':1, '_id':0}

for i in actors.find(filter, field):
    print(i)

{'배우이름': '이병헌', '출연작': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다']}
{'배우이름': '하정우', '출연작': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨']}
{'배우이름': '마동석', '출연작': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '전혜진', '출연작': ['백두산', '비스트', '뺑반', '불한당: 나쁜 놈들의 세상', '택시운전사']}
{'배우이름': '이성민', '출연작': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '김아중', '출연작': ['나쁜 녀석들: 더 무비', '더 킹']}


In [54]:
# $nor
# 흥행지수 >= 15000, 성별 = 남
match = {'$nor':[{'흥행지수':{'$lte':15000}},{'성별':'여'}]}
field = {'배우이름':1, '흥행지수':1, '_id':0,}

for i in actors.find(match, field):
    print(i)

{'배우이름': '이병헌', '흥행지수': 45760}
{'배우이름': '하정우', '흥행지수': 31124}
{'배우이름': '마동석', '흥행지수': 19019}
{'배우이름': '이성민', '흥행지수': 15213}


### 5) sortByCount
* aggregate()함수 사용

In [56]:
# 소속사별 count
match = [{'$sortByCount':"$소속사"}]
match = [{'$sortByCount':"$소속사"},{'$match':{'count':{"$gte": 4}}}]
for i in actors.aggregate(match):
    print(i)

{'_id': None, 'count': 1715}
{'_id': '아티스트컴퍼니', 'count': 16}
{'_id': '싸이더스 HQ', 'count': 11}
{'_id': '나무액터스', 'count': 10}
{'_id': 'SM엔터테인먼트', 'count': 9}
{'_id': '열음엔터테인먼트', 'count': 8}
{'_id': '㈜사람엔터테인먼트', 'count': 6}
{'_id': '키이스트', 'count': 4}
{'_id': '씨에스엑터스', 'count': 4}
{'_id': '지티비엔터테인먼트', 'count': 4}


### 6) list 검색
* 리스트 안에 들어 있는 내용을 검색하기 위해서는 순서와 값이 정확하게 일치해야 한다.
* 순서에 상관없이 값으로면 찾으려면 $all을 사용한다.

In [95]:
# 출연영화 리스트에 '반도' 를 포함하는 경우
match = {'출연작':'반도'}
# 출연영화 리스트에 '1987'또는 '반도'를 포함하는 경우
match = {'$or':[{'출연작':'1987'},{'출연작': '반도'}]}
field = {'배우이름':1, '출연작':1, '_id':0}

# 출연작 리스트에 '남산의 부장들' 과 '공작'을 포함하는 경우
match = {'출연작':['남산의 부장들', '공작']} # 조건에 만족하지 않는다.(순서도 맞아야함)
match = {'출연작':{'$all':['남산의 부장들', '공작']}} 

# 첫번째 출연작이 '결백'인 배우를 찾기
match = {'출연작.0':'결백'}

# 총 6편의 영화에 출연한 배우
match = {'출연작':{'$size': 6}}

for i in actors.find(match, field).limit(10):
    print(i)


{'배우이름': '이병헌', '출연작': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다']}
{'배우이름': '하정우', '출연작': ['백두산', '클로젯', '멋진 하루', '신과 함께-인과 연', '신과 함께-죄와 벌', '아가씨']}
{'배우이름': '마동석', '출연작': ['나쁜 녀석들: 더 무비', '악인전', '노리개: 그녀의 눈물', '부당거래', '부산행', '신과 함께-인과 연']}
{'배우이름': '이성민', '출연작': ['남산의 부장들', '비스트', '미스터 주: 사라진 VIP', '부당거래', '공작', '마이 리틀 히어로']}
{'배우이름': '박정민', '출연작': ['시동', '다만 악에서 구하소서', '타짜: 원 아이드 잭', '사냥의 시간', '그것만이 내 세상', '동주']}
{'배우이름': '조진웅', '출연작': ['광대들: 풍문조작단', '퍼펙트맨', '블랙머니', '장수상회', '사라진 시간', '명량']}
{'배우이름': '조정석', '출연작': ['엑시트', '형', '시간이탈자', '특종: 량첸살인기', '뺑반', '나의 사랑 나의 신부']}
{'배우이름': '곽도원', '출연작': ['남산의 부장들', '강철비2: 정상회담', '변호인', '아수라', '무뢰한', '범죄와의 전쟁 : 나쁜놈들 전성시대']}
{'배우이름': '유해진', '출연작': ['봉오동 전투', '부당거래', '타짜', '베테랑', '말모이', '럭키']}
{'배우이름': '류준열', '출연작': ['봉오동 전투', '리틀 포레스트', '뺑반', '돈', '택시운전사', '더 킹']}


### 7) limit()


In [88]:
# 직업이 가수이면서 흥행지수가 높은 상위 10명의 배우는?
for i in actors.find({'직업':'가수'}).sort('흥행지수', -1).limit(10):
    print(i['배우이름'], i['흥행지수'])

배수지 12646
임윤아 12343
최민호 2008
RM 1061
진 909
슈가 758
제이홉 606
준호 518
지민 455
김소혜 324


In [94]:
# '국가부도의 날' 에 출연한 배우를 흥행지수가 높은 순으로 10명 출력하기
for i in actors.find({'출연작':'국가부도의 날'},{'_id':0}).sort('흥행지수',-1).limit(10):
    print(i['배우이름'], i['흥행지수'],i['출연작'])

유아인 12422 ['＃살아있다', '베테랑', '버닝', '국가부도의 날', '우아한 거짓말']
조우진 8725 ['봉오동 전투', '내부자들', '돈', '국가부도의 날', '어쩌다, 결혼', '창궐']
허준호 6739 ['천문: 하늘에 묻는다', '퍼펙트맨', '결백', '국가부도의 날', '인랑']
전배수 1761 ['＃살아있다', '광해, 왕이 된 남자', '기묘한 가족', '국가부도의 날']
송영창 1715 ['정직한 후보', '변호인', '베테랑', '말모이', '국가부도의 날', '해빙']
김민상 121 ['남산의 부장들', '국가부도의 날', '럭키']
염혜란 90 ['야구소녀', '걸캅스', '아이 캔 스피크', '증인', '미성년', '국가부도의 날']
류덕환 46 ['난폭한 기록', '국가부도의 날', '천하장사 마돈나']
김혜수 2 ['도둑들', '타짜', '국가부도의 날']
박진주 0 ['스윙키즈', '국가부도의 날']
